In [19]:
from typing import TypedDict, Annotated, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables.base import RunnableSequence
import operator
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from langchain.agents import create_react_agent
from langchain import hub
from langgraph.graph import END, StateGraph

load_dotenv()



True

In [13]:
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


In [15]:
tools: list = [TavilySearchResults(max_results=1)]
tool_executor: ToolExecutor = ToolExecutor(tools)
prompt: PromptTemplate = hub.pull("hwchase17/react")
llm: GoogleGenerativeAI =  GoogleGenerativeAI(model="gemini-pro")
agent: RunnableSequence = create_react_agent(llm=llm, tools=tools, prompt=prompt)

In [16]:
def run_agent(data):
    agent_outcome: str = agent.invoke(data)
    return {"agent_outcome" : agent_outcome}


In [17]:
def execute_tools(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


In [18]:
def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"


In [20]:
    
# Graph app

workflow = StateGraph(AgentState)
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
workflow.add_edge("action", "agent")


In [24]:
app = workflow.compile()

In [25]:
inputs = {"input": "What is the capital of India?", "chat_history": []}

In [26]:
for stream in app.stream(inputs):
    print(list(stream.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='tavily_search_results_json', tool_input='What is the capital of India?', log='Action: tavily_search_results_json\nAction Input: What is the capital of India?')}
----
{'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='What is the capital of India?', log='Action: tavily_search_results_json\nAction Input: What is the capital of India?'), '[{\'url\': \'https://en.wikipedia.org/wiki/Delhi\', \'content\': "New Delhi was officially declared as the capital of the Union of India after the country gained independence on 15 August 1947.[55] It has expanded since; the small part of it that was constructed during the British period has come to be informally known as Lutyens\' Delhi.[56]\\nPartition and post-independence\\nDuring the partition of India, around five hundred thousand Hindu and Sikh refugees, mainly from West Punjab fled to Delhi, while around three hundred thousand Muslim residents of the city migrated to Pakistan.[5